In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# # !pip install s2sphere
from collections import defaultdict
import s2sphere as s2
from ipywidgets import interact
import requests
from io import StringIO

import dask

import time

import pandas as pd

import plotly

import plotly.graph_objs as go
import plotly.graph_objects as go

import plotly.offline as offline

from plotly.graph_objs import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from textblob import TextBlob

init_notebook_mode(connected=True)


# from dask.distributed import Client, progress
# client = Client(n_workers=4, threads_per_worker=1)
# client

# Computer BFS using Dask

In [3]:
# # small_tweets = pd.read_json("./sandy_data/tweet_ids_only.jsonl", lines=True)


# import dask.bag as db
# import json

# b = db.read_text('./sandy_data/small_tweet_ids_only.jsonl', ).map(json.loads)
# b




dask.bag<loads, npartitions=1>

In [4]:
# def flatten(record):
#     return {
#         'full_text': record['full_text'],
#         'created_at': record['created_at'],
#         'id': record['id'],
#         'long': record['coordinates']['coordinates'][0],
#         'lat': record['coordinates']['coordinates'][1],
#     }



# b.filter(lambda record: record['coordinates'] is not None).map(json.dumps).to_textfiles('processed.*.json')     # Write to local disk




['/Users/atharvas/Documents/FALL_2020/LING506/sandy/processed.0.json']

In [25]:
small_tweets = pd.read_json("./sandy_data/small_tweet_ids_only.jsonl", 'records', lines=True)


In [26]:
token = "pk.eyJ1IjoiYXRoYXJ2YXMiLCJhIjoiY2tnZ3Q2dGV4MHpreDJzdDl2Ync4ODJvZyJ9.jFeZV_Mn0i5qufHmTmYYfQ"

In [21]:

geo_tweets = small_tweets[~small_tweets['geo'].isnull()].reset_index(drop=True)
geo_tweets = geo_tweets.merge(pd.DataFrame(geo_tweets['geo'].apply(pd.Series)['coordinates'].tolist(), columns=['lat', 'long']), left_index=True, right_index=True)
geo_tweets['created_at_date'] = pd.to_datetime(geo_tweets['created_at'])
geo_tweets['created_at'] = geo_tweets['created_at'].astype(str)
# 40.7128° N, 74.0060° W
# geo_tweets = geo_tweets[(30 < geo_tweets['lat']) & (geo_tweets['lat'] < 50) & (-50 < geo_tweets['lat']) & (geo_tweets['lat'] > -85)]


get_sentiment = lambda txt : TextBlob(txt).sentiment.polarity
geo_tweets['polarity'] = geo_tweets['full_text'].apply(get_sentiment)



print(geo_tweets.columns)
geo_tweets.head(2)

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'favorited', 'retweeted',
       'possibly_sensitive', 'lang', 'retweeted_status', 'extended_entities',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'quoted_status', 'withheld_in_countries', 'lat', 'long',
       'created_at_date', 'polarity'],
      dtype='object')


,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,withheld_in_countries,lat,long,created_at_date,polarity
0,2012-10-25 19:18:02+00:00,261547183722082300,261547183722082304,So there's going to be a hurricane tonight and...,False,"[0, 129]","{'hashtags': [{'text': 'comeon', 'indices': [1...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,26.844194,-80.062265,2012-10-25 19:18:02+00:00,-0.2
1,2012-10-25 19:18:42+00:00,261547349921370100,261547349921370112,They named the hurricane that will be coming u...,False,"[0, 90]","{'hashtags': [{'text': 'wtf', 'indices': [86, ...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.618274,-74.549170,2012-10-25 19:18:42+00:00,-0.5


In [22]:
geo_tweets.to_pickle("./big.pkl")

# Viz

In [23]:
times = pd.date_range(geo_tweets['created_at_date'].min().round("1H"), geo_tweets['created_at_date'].max().round("1H") + pd.DateOffset(hours=2), freq="2H")

times = [(times[i], times[i + 1]) for i in range(len(times) - 1)]

times[0][0].strftime("%Y-%m-%d %H:%M")



process_time = lambda tup : " (" + tup[0].strftime("%Y-%m-%d %H:%M") + ", " + tup[1].strftime("%Y-%m-%d %H:%M") + ")"



In [27]:

mapbox_access_token = token

fig = go.Figure()

for start, end in times:
        selected_data = geo_tweets[(start < geo_tweets['created_at_date']) & (geo_tweets['created_at_date'] < end)]

        selected_data = selected_data[['lat', 'long', 'full_text', 'id', 'created_at', 'created_at_date', 'polarity']]

        selected_data['hover_text'] = selected_data['created_at'] + ': ' + selected_data['full_text'] 
        trace = go.Scattermapbox(
            lat=selected_data['lat'],
            lon=selected_data['long'],
            mode='markers',
            
            marker=go.scattermapbox.Marker(
                size=14, 
                color = selected_data['polarity'],
                colorscale= 'YlOrRd',
                showscale=True,
                cmax=1.0,
                cmin=-1.0),
            text=selected_data['hover_text'],
            
            name= start.strftime("%Y-%m-%d %H:%M")
        )
        
        fig.add_trace(trace)        
        
        

fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Tweets during time interval" + process_time(times[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]



fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    ), 
    sliders=sliders

)


fig.show()

In [28]:
fig.write_html("./bigplot.html")